# Fine Tuning VGG-16 as Feature Extractor 

In [1]:
# VGG-16 is applied as feature extractor by fixing all the weights and last FC layer is changed to 9-class driver 
# 17 driver RGB images
# without data augmentation 


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input



Using TensorFlow backend.


##### Loading Data 

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB/train/x_data_all.npy')    # Data Path 
y_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (17194, 224, 224, 3) y_train.shape: (17194, 9)
x_valid.shape: (2558, 224, 224, 3) y_valid.shape: (2558, 9)


In [4]:
batch_size = 64
resize = 224
n_class = 9

In [5]:
from keras.applications.vgg16 import VGG16, preprocess_input

image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')
model.summary()


W0418 19:43:36.174367 140679316420352 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:

# To get the last layer so that can be changed to our own datasets classes 

last_layer = model.get_layer('fc2').output

#x= Flatten(name='flatten')(last_layer)

out = Dense(n_class, activation='softmax', name='output')(last_layer)

vgg_model = Model(image_input, out)
vgg_model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
for layer in vgg_model.layers[:-1]:      # Freezing all the layers except the last layer
    layer.trainable = False

vgg_model.layers[3].trainable
vgg_model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Model Saving and Compling 

In [8]:

save_name = 'VGG-16_as_FeatureExtractor_200418_RGB'
save_path = './checkpoints/' + save_name

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

vgg_model = multi_gpu_model(vgg_model, gpus=2)

# aug = ImageDataGenerator(
#         rotation_range=20,
#         zoom_range=0.15,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.15,
#         horizontal_flip=True,
#         fill_mode="nearest")


# Compile CNN model
sgd = optimizers.SGD(lr = 0.001)
vgg_model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

# adam = optimizers.Adam(lr=0.0001)
# model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])
    
vgg_model.fit(x_train, y_train, 
                     batch_size=batch_size,
                    validation_data=(x_valid, y_valid),       
                    epochs=100,
                    verbose=1,
                    callbacks=callback_list)


vgg_model.save(save_path + '/epoch-finish.h5')
vgg_model.save_weights(save_path +'/epoch-finish_weights.h5')

W0418 19:43:42.320895 140679316420352 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0418 19:43:42.508856 140679316420352 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0418 19:43:42.515405 140679316420352 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0418 19:43:42.516232 140679316420352 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Train on 17194 samples, validate on 2558 samples
Epoch 1/100
17194/17194 [==============================] - 75s 4ms/step - loss: 1.2269 - accuracy: 0.6071 - val_loss: 1.5058 - val_accuracy: 0.4238

Epoch 00001: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0001.h5


W0418 19:45:43.857422 140679316420352 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/100
17194/17194 [==============================] - 68s 4ms/step - loss: 0.6005 - accuracy: 0.8385 - val_loss: 1.4282 - val_accuracy: 0.4668

Epoch 00002: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0002.h5
Epoch 3/100
17194/17194 [==============================] - 68s 4ms/step - loss: 0.4443 - accuracy: 0.8893 - val_loss: 1.4888 - val_accuracy: 0.4644

Epoch 00003: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0003.h5
Epoch 4/100
17194/17194 [==============================] - 68s 4ms/step - loss: 0.3637 - accuracy: 0.9119 - val_loss: 1.4273 - val_accuracy: 0.5039

Epoch 00004: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0004.h5
Epoch 5/100
17194/17194 [==============================] - 68s 4ms/step - loss: 0.3131 - accuracy: 0.9239 - val_loss: 1.5027 - val_accuracy: 0.4914

Epoch 00005: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0005.h5
Epoc

Epoch 35/100
17194/17194 [==============================] - 66s 4ms/step - loss: 0.0814 - accuracy: 0.9855 - val_loss: 1.6874 - val_accuracy: 0.5313

Epoch 00035: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0035.h5
Epoch 36/100
17194/17194 [==============================] - 66s 4ms/step - loss: 0.0799 - accuracy: 0.9857 - val_loss: 1.6928 - val_accuracy: 0.5266

Epoch 00036: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0036.h5
Epoch 37/100
17194/17194 [==============================] - 69s 4ms/step - loss: 0.0780 - accuracy: 0.9862 - val_loss: 1.7302 - val_accuracy: 0.5258

Epoch 00037: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0037.h5
Epoch 38/100
17194/17194 [==============================] - 67s 4ms/step - loss: 0.0765 - accuracy: 0.9871 - val_loss: 1.6992 - val_accuracy: 0.5266

Epoch 00038: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0038.h5


Epoch 68/100
17194/17194 [==============================] - 68s 4ms/step - loss: 0.0493 - accuracy: 0.9932 - val_loss: 1.7234 - val_accuracy: 0.5367

Epoch 00068: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0068.h5
Epoch 69/100
17194/17194 [==============================] - 68s 4ms/step - loss: 0.0489 - accuracy: 0.9931 - val_loss: 1.7637 - val_accuracy: 0.5340

Epoch 00069: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0069.h5
Epoch 70/100
17194/17194 [==============================] - 73s 4ms/step - loss: 0.0482 - accuracy: 0.9932 - val_loss: 1.7733 - val_accuracy: 0.5297

Epoch 00070: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0070.h5
Epoch 71/100
17194/17194 [==============================] - 73s 4ms/step - loss: 0.0477 - accuracy: 0.9935 - val_loss: 1.7875 - val_accuracy: 0.5305

Epoch 00071: saving model to ./checkpoints/VGG-16_as_FeatureExtractor_200418_RGB/checkpoint-0071.h5
